In [2]:
import numpy as np
import pandas as pd
#import shapefile
import geopandas as gpd
import matplotlib
%matplotlib inline
import json

In [3]:
#geo_data= gpd.GeoDataFrame.from_file("../Maps/BlocksQuarterMile.shp")

In [4]:
#geo_data.plot()

In [5]:
#LEHD=pd.read_csv('..\LEHD_data\LEHD_analysis.csv')
#LEHD_pluto=pd.read_csv('lehd_zip_final.csv')
#LEHD.head()
#LEHD_pluto.head()
#result = pd.merge(LEHD, LEHD_pluto, on=['C1', 'C2', 'C3', 'C4'])
#result.head()

In [6]:
pluto_df=pd.read_csv('pluto_lehd_blocks.csv')

In [7]:
pluto_df.head()

,FID,bldgar,comare,resare,retail,office,zipco,geocode,C1,C2,C3,C4
0,0,456906,54500,402406,8000,28325,11211,360470517001001,40,6,0,0
1,1,180591,65390,115201,24730,20000,11211,360470517001002,82,133,1,0
2,2,227160,92068,135092,46914,32111,11211,360470517001003,227,77,0,5
3,3,100641,15010,85631,7440,6270,11211,360470517001004,20,159,0,0
4,4,156414,54939,101475,29310,0,11249,360470517002001,55,114,0,0


# STEP 1: Coefficient calibration!

According to https://www.jtlu.org/index.php/jtlu/article/viewFile/185/175 the correlation between the density of commercial areas and the density of subway stations in NYC is equal to 0.55, while the correlation between the number of residencial areas and subway stations is equal to 0.64. We are going to use this coefficient for our estimations. 

In [8]:
T_area=np.sum(pluto_df.bldgar)
Ct_area=np.sum(pluto_df.comare)
Rt_area=np.sum(pluto_df.resare)
Ret_area=np.sum(pluto_df.retail)
coef_com=float(Ct_area)/float(T_area)
coef_res=float(Rt_area)/float(T_area)
coef_3=(float(Ct_area)+float(Ret_area))/float(T_area)
print('com_coef',coef_com)
print('res_coef',coef_res)
print('coef_3',coef_3)

('com_coef', 0.30586710847785525)
('res_coef', 0.6919942780155128)
('coef_3', 0.42572457981754663)


In [17]:
total_area = 40000000.0

In [18]:
local_gdp = Ct_area / total_area
local_gdp

0.064878000000000005

But! if we recalculate this coefficients for our area of study (quarter mile buffer around bedford av.) we got: 0.31 for commercial density, 0.69 for residential density, and 0.42 if we add retail area to commercial area. 

Even if the area is too small, this results might not be that crazy, since the paper's coefficients are calculated based on data for the entire city, Manhattan and its high density of commercial areas might skewe a little bit the coefficient. That also might explain the relatively higher coefficient for residencial areas (experimental=0.69 vs theoretical=0.64) since Williamsburgh is in general more residential than other parts of the city.

One thing to have in mind is that the area is too small (there is only one metro station!) so, trying with different areas is suggested for adjusting the coefficient. 

# STEP 2: generalization of the original equation from density of MTA stations to number of passengers

After getting the relationship between number of stations and density of commercial area, we are going to calculate a new relationship, based on ridership 

In [9]:
#first try:
#Stops considered: bedford, Lorimer, metropolitan, Nassau, Graham av
stops=['Bedford','Lorimer_Metropolitan', 'Nassau','Graham']
#annual riderships: 
riderships=[9892516, 5090344, 2760220,3402514]
daily_riders= pd.DataFrame({'stops':stops, 'users':[i/365.0 for i in riderships]})
daily_riders

,stops,users
0,Bedford,27102.783562
1,Lorimer_Metropolitan,13946.147945
2,Nassau,7562.246575
3,Graham,9321.956164


In [10]:
np.std(daily_riders.users) #STD is too high! so I can't just take the mean of riders and use it in the study, just for 
                            #the moment, I will work with the ridership of bedfor avenue 27103

7649.8422922944574

Then, our relationship between density of commercial areas and number of riders for station is going to be: 

$$dC= \alpha (R / 10,000)$$

where $dC$ is the density of commercial areas, $\alpha$ is the coefficient from the previous step and $R$ represents the ridership in the area of study.

(Note that this relationship can be generalized, taking $R$ as 'density of riders' in a given area. Since our area of study is defined as a 'unit' there is no distintion between number of riders and rider density) 

Since the relationship between Ridiership and commerce density depends a lot on local properties, we are going to calculate a coefficient $\alpha$ for each census block, instead of using a general coefficient.

In [11]:
R=daily_riders[(daily_riders['stops']=='Bedford')].users.values[0]
dC=(pluto_df['comare']+pluto_df['retail'])/pluto_df['bldgar']
pluto_df['alpha_coef']=(dC/R)*10000

In [12]:
#MAP showing alpha_coef on each census block is neccesary.

# STEP 3: estimating impact

In [ ]:
pluto_df